In [2]:
!pip install pymc3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 7.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of pymc3 to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:0000:01:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 11.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.4/115.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Created wheel for theano-pymc: filename=Theano_PyMC-1.1.2-py3-none-any.whl size=1529960 sha256=441ed44a6cefdf98fc349c1335c3253130b20f446cf379ce17222269fe622c92
  Stor

In [24]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv('medical.csv')

# Encode categorical variables
label_encoders = {}
for column in ['Disease', 'Fever', 'Cough', 'Fatigue', 'Difficulty Breathing', 'Gender', 'Blood Pressure', 'Cholesterol Level', 'Outcome Variable']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

data.head()


,Disease,Fever,Cough,Fatigue,Difficulty Breathing,Age,Gender,Blood Pressure,Cholesterol Level,Outcome Variable
0,56,1,0,1,1,19,0,1,2,1
1,24,0,1,1,0,25,0,2,2,0
2,37,0,1,1,0,25,0,2,2,0
3,6,1,1,0,1,25,1,2,2,1
4,6,1,1,0,1,25,1,2,2,1


In [26]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Define the structure of the Bayesian Network
model = BayesianNetwork([
    ('Fever', 'Disease'),
    ('Cough', 'Disease'),
    ('Fatigue', 'Disease'),
    ('Difficulty Breathing', 'Disease'),
    ('Age', 'Disease'),
    ('Gender', 'Disease'),
    ('Blood Pressure', 'Disease'),
    ('Cholesterol Level', 'Disease'),
    ('Disease', 'Outcome Variable')
])

# Parameterize the network
cpd_fever = TabularCPD(variable='Fever', variable_card=2, values=[[0.5], [0.5]])
cpd_cough = TabularCPD(variable='Cough', variable_card=2, values=[[0.5], [0.5]])
cpd_fatigue = TabularCPD(variable='Fatigue', variable_card=2, values=[[0.5], [0.5]])
cpd_difficulty_breathing = TabularCPD(variable='Difficulty Breathing', variable_card=2, values=[[0.5], [0.5]])
cpd_age = TabularCPD(variable='Age', variable_card=11, values=[[1/11], [1/11], [1/11], [1/11], [1/11], [1/11], [1/11], [1/11], [1/11], [1/11], [1/11]])
cpd_gender = TabularCPD(variable='Gender', variable_card=2, values=[[0.5], [0.5]])
cpd_blood_pressure = TabularCPD(variable='Blood Pressure', variable_card=3, values=[[1/3], [1/3], [1/3]])
cpd_cholesterol_level = TabularCPD(variable='Cholesterol Level', variable_card=3, values=[[1/3], [1/3], [1/3]])


# Define the CPD for Disease based on all parent nodes
# This should be learned from the data. For simplicity, assume equal probabilities here
# Define the CPD for Disease based on all parent nodes
# Define the CPD for Disease based on all parent nodes
cpd_disease = TabularCPD(
    variable='Disease', variable_card=10,
    values=[[1/10]*3168] * 10,
    evidence=['Fever', 'Cough', 'Fatigue', 'Difficulty Breathing', 'Age', 'Gender', 'Blood Pressure', 'Cholesterol Level'],
    evidence_card=[2, 2, 2, 2, 11, 2, 3, 3]  # Adjust with actual cardinalities
)

# Define the CPD for Outcome Variable based on Disease
cpd_outcome = TabularCPD(
    variable='Outcome Variable', variable_card=2,
    values=[[0.5]*10, [0.5]*10],
    evidence=['Disease'],
    evidence_card=[10]  # Adjust with the actual cardinality of 'Disease'
)


# Add CPDs to the model
model.add_cpds(cpd_fever, cpd_cough, cpd_fatigue, cpd_difficulty_breathing, cpd_age, cpd_gender, cpd_blood_pressure, cpd_cholesterol_level, cpd_disease, cpd_outcome)

# Check if the model is valid
model.check_model()

# Perform exact inference
infer = VariableElimination(model)
posterior_prob = infer.query(['Outcome Variable'], evidence={'Fever': 1, 'Cough': 1})
print(posterior_prob)


+---------------------+-------------------------+
| Outcome Variable    |   phi(Outcome Variable) |
+=====================+=========================+
| Outcome Variable(0) |                  0.5000 |
+---------------------+-------------------------+
| Outcome Variable(1) |                  0.5000 |
+---------------------+-------------------------+


In [29]:
import matplotlib.pyplot as plt

# Perform exact inference
infer = VariableElimination(model)

# Predict diseases for each instance in the dataset
predicted_diseases = []
for index, instance in data.iterrows():
    evidence = instance.to_dict()
    del evidence['Outcome Variable']  # Remove Outcome Variable from evidence if it exists in your dataset
    
    # Perform inference to get posterior probabilities of 'Disease'
    posterior_prob = infer.query(['Disease'], evidence=evidence)
    
    # Get the predicted disease (highest probability)
    predicted_disease = posterior_prob.values.argmax()
    predicted_diseases.append(predicted_disease)

# Add predicted diseases back to the dataframe for visualization
data['Predicted Disease'] = predicted_diseases

# Visualize the predicted diseases
plt.figure(figsize=(10, 6))
data['Disease'].value_counts().plot(kind='bar', color='blue', alpha=0.5, label='Actual Diseases')
data['Predicted Disease'].value_counts().plot(kind='bar', color='red', alpha=0.5, label='Predicted Diseases')
plt.title('Actual vs Predicted Diseases')
plt.xlabel('Disease Index')
plt.ylabel('Count')
plt.legend()
plt.show()


ValueError: Can't have the same variables in both `variables` and `evidence`. Found in both: {'Disease'}